# Create small netcdf
We import the arrays generated in `firesmoke_make_metadata.ipynb` as well as additional metadata from original netCDF file.

## Import necessary libraries

In [1]:
# to open original netcdfs and new netcdf with idx url
import xarray as xr
import numpy as np
# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy, 
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

## Load the original metadata from one dispersion.nc file

Get path to a netcdf file containing metadata

In [2]:
# ******* THIS IS WHEN RUNNING FROM ATLANTIS.SCI **************
firesmoke_file = "/usr/sci/cedmav/data/firesmoke/BSC00CA12-01/dispersion_20240103.nc"

In [3]:
# load the original netcdf file
ds = xr.open_dataset(firesmoke_file)

# take a look
ds

<xarray.Dataset> Size: 84MB
Dimensions:  (TSTEP: 51, VAR: 1, DATE-TIME: 2, LAY: 1, ROW: 381, COL: 1081)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 408B ...
    PM25     (TSTEP, LAY, ROW, COL) float32 84MB ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2024003
    CTIME:          120044
    WDATE:          2024003
    ...             ...
    VGLVLS:         [10.  0.]
    GDNAM:          HYSPLIT CONC    
    UPNAM:          hysplit2netCDF  
    VAR-LIST:       PM25            
    FILEDESC:       Hysplit Concentration Model Output                       ...
    HISTORY:

Drop the TFLAGs and create a new TFLAGs variable with all 27k timesteps. 
Drop PM25 variable, this is the variable of interest and already in IDX file.

In [4]:
# drop the TFLAG variable and add our own, also drop PM25, this is the array stored by idx
# ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.drop_vars.html
ds_new = ds.drop_vars("TFLAG")
ds_new = ds_new.drop_vars("PM25")

# ds_new is now empty except for attributes
ds_new

<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/33)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2024003
    CTIME:          120044
    WDATE:          2024003
    ...             ...
    VGLVLS:         [10.  0.]
    GDNAM:          HYSPLIT CONC    
    UPNAM:          hysplit2netCDF  
    VAR-LIST:       PM25            
    FILEDESC:       Hysplit Concentration Model Output                       ...
    HISTORY:

## Create new variables for metadata from `firesmoke_make_metadata.ipynb` and add to our netCDF file

In [5]:
def add_var(ds, metadata, attr_name):
    '''
    Add the metadata given to the xarray DataSet, using attr_name
    '''
    # get dimensions currently in ds
    dims = [*ds.dims]
    
    # append new dimension to dims
    dims.append(attr_name)
    
    # create a new DataArray using metadata
    # ref: https://docs.xarray.dev/en/latest/generated/xarray.Variable.html
    # ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html
    new_var = xr.DataArray(metadata, dims=dims)
    
    # add metadata to dataset
    ds_new[attr_name] = new_var

In [ ]:
# import metadata
tflags = np.load("firesmoke_v4-tflags.npy")
init_tflags = np.load("firesmoke_v4-last_init.npy")
resamp = np.load("firesmoke_v4-resamp.npy")

# reshape timeflags so they're exactly like UBC's original tflags
tflags = np.expand_dims(tflags, axis=1)
init_tflags = np.expand_dims(init_tflags, axis=1)

# make timeflags int32, as in original netcdf
tflags = tflags.astype(np.int32)
init_tflags = init_tflags.astype(np.int32)

# add metadata to dataset
add_var(

# take a look, now we just need to add pm25
ds_new

In [ ]:
# Create an PM25 dataarray representing dims of one time slice
pm25_var = xr.DataArray(np.zeros((ds.NROWS, ds.NCOLS)), dims=['ROW', 'COL'], attrs=ds['PM25'].attrs)

# add PM25 to dataset, this is strictly for storing the dimensionality of PM25
# as this is dimensionality info is not available to xarray from idx file
ds_new['PM25'] = pm25_var

In [ ]:


# make them int32, as in original netcdf
init_tflags = init_tflags.astype(np.int32)

# replace 'TSTEP' with time, to be compatible with our openvisus backend_v3.py
dims = [*ds['TFLAG'].dims]
dims[0] = 'time'

# create a new DataArray using imported tflags, use same attributes and dims from original netcdf file
# ref: https://docs.xarray.dev/en/latest/generated/xarray.Variable.html
# ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html
tflags_var = xr.DataArray(tflags, dims=dims, attrs=ds['TFLAG'].attrs)

# add tflags to dataset
ds_new['TFLAG'] = tflags_var

# take a look, now we just need to add pm25
ds_new

### Add idx_url as an attribute, then we're ready to save our tiny netcdf.

In [ ]:
# url to idx file
idx_url = "http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1"

# Adding a new idx_url attribute
ds_new = ds_new.assign_attrs({'idx_url': idx_url})

In [ ]:
# directory to new tiny netcfd file
new_nc = "firesmoke_metadata.nc"

# save xarray dataset as netcdf, this is the metadata we put into tiny netcdf
ds_new.to_netcdf("firesmoke_metadata.nc")

In [ ]:
# check out the tiny netcdf loaded by vanilla xarray
xr.open_dataset(new_nc)

## Test tiny netcdf

## Load tiny netcdf file with backend_v3 and pass timestamps above as timesteps

In [ ]:
# load data
final_ds = xr.open_dataset(new_nc, engine=OpenVisusBackendEntrypoint)
final_ds